In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [37]:
df_distance_km = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')
df_location = pd.read_excel('../../Datos_P1/df_location.xlsx')
df_orders = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_vehicle = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')

In [38]:
clientes_dict = {
    row['Cliente']: {
        'latitud': row['Latitud'],
        'longitud': row['Longitud']
    }
    for _, row in df_location.iterrows()
}

ordenes_dict = {
    row['cliente']: row['order_demand']
    for _, row in df_orders.iterrows()
}

vehiculos_dict = {
    int(row['vehiculo_id']): {
        'capacidad': float(row['capacidad_kg']),
        'autonomia': float(row['autonomia_km']),
        'coste_km': float(row['costo_km'])
    }
    for _, row in df_vehicle.iterrows()
}

In [39]:

Almacen = (-3.7270384, 40.3885963)  #Localización del almacén hardcoded, tengo que cambiar esto si lo usamos

# client_positions = []
# for i, row in df_location.iterrows():
#     client_positions.append(dict(f'Cliente {i}',(row['Longitud'], row['Latitud'])))

# Definir la matriz de distancias entre clientes
distances = df_distance_km.to_numpy()


# Datos de ejemplo
clientes = list(range(1, 21))  # Clientes 1 a 20
random.shuffle(clientes)

# Función para generar una solución aleatoria
def generar_soluciones_aleatorias(clientes, vehiculos, ordenes_dict, distancias, numero_rutas, random_state=None):
    if random_state is not None:
        random.seed(random_state)  # Fijamos el estado aleatorio
        np.random.seed(random_state)

    rutas_creadas = []

    for _ in range(numero_rutas):
        # Crear una lista vacía para las rutas de los vehículos
        rutas = {vehiculo: [0] for vehiculo in vehiculos}  # Iniciar con el almacén (0)
        clientes_disponibles = clientes.copy()  # Lista de clientes por asignar
        
        # Asignar clientes aleatoriamente a los vehículos, respetando la capacidad
        for cliente in clientes_disponibles:
            asignado = False
            while not asignado:
                cliente_str = f"Cliente_{cliente}"
                vehiculo_id = random.choice(list(vehiculos.keys()))  # Escoge un vehículo aleatorio

                ultima_parada = rutas[vehiculo_id][-1]
                if ultima_parada == 0:
                    ultima_parada = 21  # El almacén está en el índice 20  

                capacidad_usada = sum([ordenes_dict[f"Cliente_{c}"] for c in rutas[vehiculo_id] if c != 0]) + ordenes_dict[cliente_str]
                if capacidad_usada <= vehiculos[vehiculo_id]['capacidad'] and distancias[ultima_parada-1][cliente-1] > 0:
                    rutas[vehiculo_id].append(cliente)  # Asignar el cliente a este vehículo
                    asignado = True
        
        # Asegurar que cada vehículo termine en el almacén (0)
        for vehiculo in rutas:
            rutas[vehiculo].append(0)  # Añadir el almacén al final de la ruta de cada vehículo

        capacidad_por_vehiculo = {}
        for vehiculo, ruta in rutas.items():
            capacidad_usada = sum([ordenes_dict[f"Cliente_{c}"] for c in ruta if c != 0])  # Sumar solo los clientes (excluyendo el almacén)
            capacidad_por_vehiculo[vehiculo] = capacidad_usada

        rutas_creadas.append((rutas))

    return rutas_creadas


 #Generar una solución aleatoria
rutas = generar_soluciones_aleatorias(clientes, vehiculos_dict, ordenes_dict, distances, 5, 1337)
for i, (rutas) in enumerate(rutas):
    print(f"Solución {i + 1}:")
    
    # Recorrer cada vehículo y su ruta
    for vehiculo, ruta in rutas.items():
         # Mostrar la ruta y la capacidad utilizada por cada vehículo
        print(f"  Vehículo {vehiculo}: Ruta -> {ruta}")

Solución 1:
  Vehículo 1: Ruta -> [0, 13, 1, 0]
  Vehículo 2: Ruta -> [0, 14, 8, 12, 0]
  Vehículo 3: Ruta -> [0, 18, 2, 10, 11, 15, 0]
  Vehículo 4: Ruta -> [0, 9, 6, 16, 0]
  Vehículo 5: Ruta -> [0, 20, 5, 4, 3, 0]
  Vehículo 6: Ruta -> [0, 19, 7, 17, 0]
Solución 2:
  Vehículo 1: Ruta -> [0, 18, 17, 0]
  Vehículo 2: Ruta -> [0, 5, 13, 16, 8, 0]
  Vehículo 3: Ruta -> [0, 20, 1, 3, 0]
  Vehículo 4: Ruta -> [0, 10, 12, 0]
  Vehículo 5: Ruta -> [0, 19, 14, 2, 9, 6, 4, 0]
  Vehículo 6: Ruta -> [0, 7, 11, 15, 0]
Solución 3:
  Vehículo 1: Ruta -> [0, 20, 2, 0]
  Vehículo 2: Ruta -> [0, 7, 6, 13, 12, 0]
  Vehículo 3: Ruta -> [0, 19, 11, 0]
  Vehículo 4: Ruta -> [0, 18, 3, 0]
  Vehículo 5: Ruta -> [0, 5, 14, 9, 15, 16, 1, 8, 0]
  Vehículo 6: Ruta -> [0, 17, 10, 4, 0]
Solución 4:
  Vehículo 1: Ruta -> [0, 7, 16, 0]
  Vehículo 2: Ruta -> [0, 1, 8, 4, 0]
  Vehículo 3: Ruta -> [0, 10, 15, 13, 12, 0]
  Vehículo 4: Ruta -> [0, 18, 2, 3, 0]
  Vehículo 5: Ruta -> [0, 19, 14, 9, 11, 6, 0]
  Vehículo 6

In [40]:
def evaluar_solucion(rutas, vehiculos, distancias, ordenes_dict):
    """
    Función para evaluar la solución según el coste total (distancia recorrida y coste por kilómetro).
    Penaliza si algún vehículo excede su capacidad.
    """
    coste_total = 0  # Inicializamos el coste total
    penalizacion_capacidad = 0  # Penalización por capacidad excedida
    penalizacion_duplicados = 0 # Penalización por duplicar clientes en la misma ruta
    penalizacion = 1e6  # Penalización alta para soluciones inválidas.
    clientes_en_rutas = set()

    # Recorremos cada vehículo y su ruta
    for vehiculo, ruta in rutas.items():
        clientes_visitados = [cliente for cliente in ruta if cliente != 0]
        duplicados = len(clientes_visitados) - len(set(clientes_visitados))

        if duplicados > 0:
            penalizacion_duplicados += duplicados * 1000  # Penalización ajustable por cliente duplicado

        capacidad_utilizada = sum([ordenes_dict[f"Cliente_{c}"] for c in ruta if c != 0])  # Sumar las órdenes de los clientes

        # Si la capacidad es excedida, aplicar una penalización
        if capacidad_utilizada > vehiculos[vehiculo]['capacidad']:
            penalizacion_capacidad += (capacidad_utilizada - vehiculos[vehiculo]['capacidad']) * penalizacion  # Penalización ajustable

        # Calcular la distancia recorrida por el vehículo
        distancia_recorrida = 0
        for i in range(len(ruta) - 1):
            cliente_actual = ruta[i]
            cliente_siguiente = ruta[i + 1]

            # Si el cliente es el almacén (0), lo mapeamos correctamente al final de la matriz
            if cliente_actual == 0:
                cliente_actual = 21  # El almacén está en el índice 20
            if cliente_siguiente == 0:
                cliente_siguiente = 21  # El almacén está en el índice 20

            # Sumar la distancia entre el cliente actual y el siguiente (ajustando para el almacén)
            distancia_recorrida += distancias[cliente_actual-1][cliente_siguiente-1]

        # Calcular el coste de la ruta (distancia * coste por kilómetro)
        coste_ruta = distancia_recorrida * vehiculos[vehiculo]['coste_km']
        coste_total += coste_ruta  # Acumular el coste de la ruta

    # Aplicar penalización por capacidad excedida
    coste_total += penalizacion_capacidad


    #print(f"El coste total de la solución es: {coste_total}")
    return coste_total

# # Evaluar la solución (con n_clientes = 4)
# coste = evaluar_solucion(rutas, vehiculos_dict, distances, ordenes_dict)
# print(f"El coste total de la solución es: {coste}")

In [41]:
def mutacion(individuo, vehiculos, random_state=None):
    if random_state is not None:
        random.seed(random_state)  # Fijamos el estado aleatorio
        np.random.seed(random_state)

    vehiculo = random.choice(list(vehiculos.keys()))
    ruta = individuo[vehiculo]  # Accedemos solo a la ruta de ese vehículo
    
    # Realiza una mutación sobre la ruta (por ejemplo, intercambiar dos clientes)
    if len(ruta) > 2:
        i, j = random.sample(range(1, len(ruta) - 1), 2)  # Evita el almacén
        ruta[i], ruta[j] = ruta[j], ruta[i]
    
    individuo[vehiculo] = ruta  # Actualizamos la ruta en el individuo
    return individuo

def seleccion_torneo(poblacion, k=3, random_state=None):
    if random_state is not None:
        random.seed(random_state)  # Fijamos el estado aleatorio
        np.random.seed(random_state)

    padres = []
    while len(padres) < 2:  # Queremos seleccionar dos padres
        # Seleccionar aleatoriamente 'tamano_torneo' individuos de la población
        torneo = random.sample(poblacion, k)

        # Evaluar el fitness de estos individuos
        fitness_torneo = [individuo[1] for individuo in torneo]

        # Seleccionar el mejor individuo según el fitness (en este caso, el de menor coste)
        ganador = torneo[fitness_torneo.index(min(fitness_torneo))]

        # Añadir al padre seleccionado
        padres.append(ganador[0])

    return padres

def crossover(ruta1, ruta2, vehiculos, random_state=None):
    if random_state is not None:
        random.seed(random_state)
        np.random.seed(random_state)
    
    hijo1 = {}
    hijo2 = {}
    
    for vehiculo in vehiculos:
        # Verificar que las rutas no estén vacías
        if len(ruta1[vehiculo]) > 1 and len(ruta2[vehiculo]) > 1:
            # Asegurarse de que las rutas tengan más de un cliente (evitar cortar solo el almacén)
            punto_corte = random.randint(1, min(len(ruta1[vehiculo]), len(ruta2[vehiculo])) - 1)
            
            # Realizar el cruce de las rutas para cada vehículo
            hijo1[vehiculo] = ruta1[vehiculo][:punto_corte] + ruta2[vehiculo][punto_corte:]
            hijo2[vehiculo] = ruta2[vehiculo][:punto_corte] + ruta1[vehiculo][punto_corte:]
        else:
            # Si alguna de las rutas tiene solo el almacén, no se hace cruce y se copia la ruta tal cual
            hijo1[vehiculo] = ruta1[vehiculo]
            hijo2[vehiculo] = ruta2[vehiculo]
    
    return hijo1, hijo2


In [42]:
def algoritmo_genetico(clientes, vehiculos, ordenes_dict, distancias, N=50, generaciones=100, k=3, tasa_crossover=0.8, tasa_mutacion=0.2, random_state=None):
    if random_state is not None:
        random.seed(random_state)  # Fijamos el estado aleatorio
        np.random.seed(random_state)
        
    # Paso 1: Inicialización de la población
    poblacion = generar_soluciones_aleatorias(clientes, vehiculos, ordenes_dict, distancias, N, random_state)
    poblacion_con_costes = [(individuo, evaluar_solucion(individuo, vehiculos, distancias, ordenes_dict)) for individuo in poblacion]

    # Paso 2: Evolución de la población
    for generacion in range(generaciones):
        nueva_poblacion = []
        
        # Selección, crossover y mutación
        while len(nueva_poblacion) < N:
            # Selección de dos padres
            padres = seleccion_torneo(poblacion_con_costes, k, random_state)
            padre1 = padres[0]
            padre2 = padres[1]
            # Crossover
            if random.random() < tasa_crossover:
                hijo1, hijo2 = crossover(padre1, padre2, vehiculos, random_state)
                nueva_poblacion.append(hijo1)
                nueva_poblacion.append(hijo2)
            else:
                nueva_poblacion.append(padre1)
                nueva_poblacion.append(padre2)
            
            # Mutación
            if random.random() < tasa_mutacion:
                hijo_mutado = mutacion(nueva_poblacion[-1], vehiculos, random_state)
                nueva_poblacion[-1] = hijo_mutado

        # Evaluar nueva población
        poblacion_con_costes = [(individuo, evaluar_solucion(individuo, vehiculos, distancias, ordenes_dict)) for individuo in nueva_poblacion]
        
        # Ordenar población por coste
        poblacion_con_costes.sort(key=lambda x: x[1])  # Ordenar por el coste de menor a mayor
        
        # Imprimir el progreso
        #print(f"Generación {generacion + 1} - Mejor coste: {poblacion_con_costes[0][1]}")

    # Resultado final
    mejor_solucion = poblacion_con_costes[0]
    return mejor_solucion[0], mejor_solucion[1]

In [43]:

mejor_ruta, mejor_coste = algoritmo_genetico(clientes, vehiculos_dict, ordenes_dict, distances, N=10, generaciones=20, random_state=1234)
print("Mejor ruta:", mejor_ruta)
print("Mejor coste:", mejor_coste)

Mejor ruta: {1: [0, 2, 2, 0], 2: [0, 10, 4, 0], 3: [0, 9, 6, 15, 13, 16, 0], 4: [0, 20, 7, 17, 0], 5: [0, 18, 14, 11, 12, 3, 0], 6: [0, 5, 18, 14, 0]}
Mejor coste: 46.693119
